In [1]:
from crimson.executable_types_beta.mock import TensorMock
from crimson.executable_types_beta.types import *
from inspect import currentframe

In [2]:
from crimson.executable_types_beta.executor import AnnotationExecutor, with_executable

If the `func_with_error` function is executed,
We get `comparison expression`s or `argument`s.

Or, an argument is assigned more than once, it is `tested` immediately.

- array1
	- b = 4,
		- type: argument
	- c = 6,
		- type: argument
- array2
	- d = 8
		- type: argument
	- e = 10
		- type: argument
	- d == 2*b
		- type: comparison
	- e == b+2*c
		- type: comparison
- array3
	- None
- array4
	- b == 4
		- type: tested
	- f = 8
		- type: argument
	- f == 3 * b
		- type: comparison

We intentionally added wrong comparisons. Let's see the result.


In [3]:
array1 = TensorMock.ones((4, 6))
array2 = TensorMock.ones((8, 10))
array3 = TensorMock.ones((4, 8))
array4 = TensorMock.ones((4, 8))

In [4]:
@AnnotationExecutor.with_executable
def func_with_error(
    array1: TensorMock[List, "b", "c"], 
    array2: TensorMock[List, "d == 2*b", "e == b+2*c"],
    array3: TensorMock
) -> TensorMock[List, "b", "f == 3 * b"]:
    return array4

In [5]:
AnnotationExecutor.set_config(
    mode='on',
    raise_error=False,
    push_result=False
)

In [6]:
AnnotationExecutor.env = currentframe().f_locals# {'TensorMock':TensorMock}

func_with_error(
    array1,
    array2,
    array3
)

[[1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]

In [7]:
AnnotationExecutor.last_execution_result.test_result

{'d == 2 * b': True, 'e == b + 2 * c': False, 'f == 3 * b': False}

If you run the same function, but with `raise_error = True`,
the execution of the function will raise an error after it is tested.


In [8]:
AnnotationExecutor.set_config(
    raise_error=True,
)

In [9]:
try:
	func_with_error(
		array1,
		array2,
		array3
	)
except Exception as e:
	print(e)

test_meta_dict failed. Check fields in AnnotationExecutor.test_result.


To prevent the memory leaking,
the default value of `push_result` is `False`.

If means that, when `AnnotationExecutor` tests various functions,
the `test_result` is replaced by each test, and previous ones are not stored.

If you want to collect them, set the value to be `True`.

In [10]:
AnnotationExecutor.execution_results

{}

In [11]:
AnnotationExecutor.set_config(
    raise_error=False,
    push_result=True
)

func_with_error(
	array1,
	array2,
	array3
)

[[1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]

In [12]:
AnnotationExecutor.execution_results

{'func_with_error': ExecutionResult(custom_meta={}, args_detail={'array1': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], 'array2': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'array3': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'return': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}, meta_annotations={'array1': ['b', 'c'], 'array2': ['d == 2*b', 'e == b+2*c'], 'array3': None, 'return': ['b', 'f == 3 * b']}, meta_dict={'b': 4, 'c': 6, 'd': 8, 'e': 10, 'f': 8}, test_result={'d == 2 * b': True, 'e == b + 2 * c': False, 'f == 3 * b': False})}

To understand the fields of the result, read the #study_module page.

Turn off the mode if you don't want to use the `AnnotationExecutor`.


In [13]:
del AnnotationExecutor.last_execution_result

In [14]:
AnnotationExecutor.set_config(
    mode='off'
)


In [15]:
@AnnotationExecutor.with_executable
def func_with_error(
    array1: TensorMock[List, "b", "c"], 
    array2: TensorMock[List, "d == 2*b", "e == b+2*c"],
    array3: TensorMock
) -> TensorMock[List, "b", "f == 3 * b"]:
    return array4

In [16]:
hasattr(AnnotationExecutor, 'last_execution_result')

False